In [1]:
#Basic download radiosonde data from Wyoming Upper Air Sounding
import urllib3
from bs4 import BeautifulSoup
import numpy as np
#sample input values for Kolkata Dumdum station for 12th Jan 2019 00Z
stn = '42809' 
year= '2019'
month = '01'
day = '12'
#enter Zulu time (Z) i.e. UTC e.g. 12 or 00
hour = '00'  
# The file will be saved at same directory as this notebook by default else provide path.
path=''
# 1)
# Wyoming URL to download Sounding from
http = urllib3.PoolManager()
#you need to modify below provided you know how the web address look generically, where the queries are fit.
url = 'http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR='+year+'&MONTH='+month+'&FROM='+day+hour+'&TO='+day+hour+'&STNM='+stn
response = http.request('GET', url)
#content = urllib2.urlopen(url).read()

# 2)
# Remove the html tags
soup = BeautifulSoup(response.data,"lxml")
data_text = soup.get_text()

# 3)
# Split the content by new line.
splitted = data_text.split("\n",data_text.count("\n"))
rdata=list('')
#end=len(splitted) #end of file
end=splitted.index('Station information and sounding indices'); #where station info & sounding info starts. 
for i in list(range(9, end)):
        splitagain=list(filter(None,splitted[i].split(" ",splitted[i].count(" "))))
        l=len(splitagain) #no of cols with data
        if (l==11):
            rdata.append(splitagain)
rawdata=np.array(rdata)
# 4)
# Write this splitted text to a .txt document
Sounding_filename =str(path)+str(stn)+'.'+str(year)+str(month)+str(day)+str(hour)+'.txt'
np.savetxt(Sounding_filename, rawdata, delimiter=' ', fmt='%s')
f=open(Sounding_filename,'a')
# 5) 
# To append station & sounding info at the end uncomment below line
#np.savetxt(f, splitted[end:(end+28)], delimiter=' ', fmt='%s') 
print("\nSuccess!",rawdata.shape,"sounding data saved to",Sounding_filename,"\n")
f.close()


Success! (93, 11) sounding data saved to 42809.2019011200.txt 



In [2]:
#This code-block takes user input for stations,years,months,days and hours to download
#Suppose we wish to download only 1-9th Sept 2022 00z and 12z at VECC/Dumdum Kolkata and VABP/Raja Bhoj Airport Bhopal
#/*Enter station numbers:
#42809 42667
#Enter year or start and end years:
#2022
#Enter month or start and end months:
#9
#Enter day or start and end days:
#1 9
#Enter Z-hour or multiple Z-hours:
#00 12*/

import urllib3
from bs4 import BeautifulSoup
import numpy as np
from http.client import responses
stn = [int(x) for x in input("Enter station numbers:\n").split()]
year=[int(x) for x in input("Enter year or start and end years:\n").split()]
year=list(range(year[0],year[len(year)-1]+1))
month=[int(x) for x in input("Enter month or start and end months:\n").split()]
month=list(range(month[0],month[len(month)-1]+1))
day=[int(x) for x in input("Enter day or start and end days:\n").split()]
day=list(range(day[0],day[len(day)-1]+1))
hour=[str(x) for x in input("Enter Z-hour or multiple Z-hours:\n").split()]
# if user wants to download specific dates only try defining by hand as list and copying in a block below -run it instead of this.
# e.g. 
#stn = [42809,42667]
#year=[2022]
#month=[9]
#day=[1,2,3,4,5,6,7,8,9]
#hour=['00','12']

Enter station numbers:
42809 42667
Enter year or start and end years:
2022
Enter month or start and end months:
9
Enter day or start and end days:
1 9
Enter Z-hour or multiple Z-hours:
00 12


In [3]:
#Below is the multiple download engine. Run this code-block only after above or atleast define stn, year, month, day, hour as list in a block above.
#This downloads as many files as user input without concatenation grouped by months
#Enter path to the directories of MON/ i.e. JAN/ FEB/ MAR/ where files would be saved...
path='' 
count=0
for ind1 in range(0,len(stn)):
    for ind2 in range(0,len(year)):
        for ind3 in range(0,len(month)):
            for ind4 in range(0,len(day)):   
                for ind5 in range(0,len(hour)):
                    http = urllib3.PoolManager()
                    url = 'http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR='+str(year[ind2])+'&MONTH='+str(month[ind3])+'&FROM='+str(day[ind4])+str(hour[ind5])+'&TO='+str(day[ind4])+str(hour[ind5])+'&STNM='+str(stn[ind1])
                    response = http.request('GET', url)
                    http_status = responses[response.status]
                    if http_status!='OK':
                        print("Cannot download",stn[ind1],year[ind2],"mon",month[ind3],"dd",day[ind4],"hh",hour[ind5],"\n")
                        continue
                    else:
                        print("Downloading from",url,stn[ind1],year[ind2],"mon",month[ind3],"dd",day[ind4],"hh",hour[ind5],"\n")
                        soup = BeautifulSoup(response.data,"lxml")
                        data_text = soup.get_text()
                        splitted = data_text.split("\n",data_text.count("\n"))
                        rdata=list('')
                        strsp='Station information and sounding indices';
                        if strsp not in splitted:
                            print("Wyoming website reports: 'cannot get observations'","\n")
                            continue                           
                        end=splitted.index('Station information and sounding indices');
                        for i in list(range(9, end)):
                            splitagain=list(filter(None,splitted[i].split(" ",splitted[i].count(" "))))
                            l=len(splitagain)
                            if (l==11):
                                rdata.append(splitagain)
                        rawdata=np.array(rdata)
# Attention! Create folders in mentioned path for atleast those months immediately downloading; spell them same as below like JAN,FEB,MAR... 
                        mon=['JAN/','FEB/','MAR/','APR/','MAY/','JUN/','JUL/','AUG/','SEP/','OCT/','NOV/','DEC/'] #create folders of same name in the path first.
                        Sounding_filename =str(path)+str(mon[month[ind3]-1])+str(stn[ind1])+'.'+str(year[ind2])+'.'+str(month[ind3])+'.'+str(day[ind4])+'.'+str(hour[ind5])+'.txt'
                        print(Sounding_filename)
                        np.savetxt(Sounding_filename, rawdata, delimiter=' ', fmt='%s')
                        f=open(path+'Sounding_info&indices','a')
                        l=list('')
                        l.append(splitted[3])
                        l.append(splitted[end:(end+28)])
                        l.append('\n')
                        # Attention! To save all station & sounding info of the end in the path/Soustation & sounding infonding_info&indices file uncomment below line
                        #np.savetxt(f,l, delimiter=' ', fmt='%s')
                        f.close()
                        count=count+1
                        print("Success!\n")
print("\n Finished downloading",count,"files")


SEP/42809.2022.9.1.00.txt
Success!


SEP/42809.2022.9.1.12.txt
Success!


SEP/42809.2022.9.2.00.txt
Success!


SEP/42809.2022.9.2.12.txt
Success!


SEP/42809.2022.9.3.00.txt
Success!


SEP/42809.2022.9.3.12.txt
Success!


SEP/42809.2022.9.4.00.txt
Success!


SEP/42809.2022.9.4.12.txt
Success!


SEP/42809.2022.9.5.00.txt
Success!


SEP/42809.2022.9.5.12.txt
Success!


SEP/42809.2022.9.6.00.txt
Success!


SEP/42809.2022.9.6.12.txt
Success!


SEP/42809.2022.9.7.00.txt
Success!


SEP/42809.2022.9.7.12.txt
Success!


SEP/42809.2022.9.8.00.txt
Success!


SEP/42809.2022.9.8.12.txt
Success!


SEP/42809.2022.9.9.00.txt
Success!

Cannot download 42809 2022 mon 9 dd 9 hh 12 


SEP/42667.2022.9.1.00.txt
Success!


Wyoming website reports: 'cannot get observations' 


SEP/42667.2022.9.2.00.txt
Success!


Wyoming website reports: 'cannot get observations' 


SEP/42667.2022.9.3.00.txt
Success!


Wyoming website reports: 'cannot get observations' 


SEP/42667.2022.9.4.00.txt
Success!


Wyoming websit

In [17]:
# Concatenate files
import os
count=0
# Path to where the files are saved . Default assumes files in current directory.
path='SEP/';
# Give concatenated file a name.
filename='concat.txt'
# Path to where to save the concatenated file. Default assumes files in current directory. 
pathsave=''
namelist=[f for f in os.listdir(path) if f.endswith('.txt')]
file=open(pathsave+filename,"a+")      
for all_files in namelist:
    temp=open(path+all_files,"r")
    f1=temp.readlines()
    count=count+1
    for x in f1:
        file.write(x) #readline by line and write to file!
    temp.close()
file.close()
print("\nSuccess! Concatenated \n",count," files to",filename)


Success! Concatenated 
 26  files to concat.txt
